# ACSE-7 (Inversion and Optimisation)

# Homework Lecture 8: Data Assimilation

### Probabilities

1. Show that given $n$ independent trials ${X_1, \ldots X_n}$ taken from a random variable $X$, with mean $\mu$ and variance $\sigma^2$, we have the result
$$ E\left(\left[\sum_i X_i^2\right]-\frac{1}{n}\left[\sum_i X_i\right]^2\right) = (n-1)\sigma^2.$$
You may use the identities that $\left(\sum_{i=1}^n a_i\right)^2=\sum_{i=1}^n a_i^2+\sum_{i=1}^n \sum_{j\neq i}  a_i a_j$, that $E(X+Y) = E(X)+E(Y)$ and that _for independent variables_ $E(XY) = E(X)E(Y)$. Remember the definitions $E(X_i)=\mu$ and $E(X_i^2)-\mu^2=\sigma^2$ and that for a constant, $\sum_{c=1}^nc= nc$.  

2.  Prove the identity for covariances that 
$$ E \left( [X-\mu_X][Y-\mu_Y]\right) = E(XY)-\mu_X\mu_Y.$$
You may use the facts that $\mu_X:=E(X)$ and $\mu_Y:=E(Y)$ are constants, that $E(aX) = aE(X)$ and that $E(a) = a$ for constant variables, as well as the identities given in question 1.
3. The [polar method](https://apps.dtic.mil/dtic/tr/fulltext/u2/288931.pdf) to generate Gaussian random variables can be stated as follows:
   1. Take a pair $(u,v)$ of uniform variables over $[-1,1]$.
   2. Let $s=u^2+v^2$. If $s=0$ or $s>1$ goto A and try again.
   3. Otherwise, calculate 
   $$x=u\sqrt{\frac{-2\ln s}{s}}, \quad y=v\sqrt{\frac{-2\ln s}{s}}.$$ These will be normal random variables of zero mean and unit variance.
   
   Implement this method in Python. Plot a histogram of your results to confirm you have the correct distribution.
 Can you extend the method to work for arbitrary means and variances?


### The BLUE

 1. Try to extend the BLUE analysis to the case of three independent unbiased observations, then  optionally generalize it for $n$ observations. You can do this either using substitution (as in the lecture notes), or by using a Lagrange multiplier for the constraint $E(\epsilon_\mbox{guess})=0$.
 2. The BLUE is the best (in the sense of minimum mean square error) linear unbiased estimator. Treating $T_t$ as a constant deterministic variable, can you find a better biased linear estimator, specified in terms of the $\sigma_i^2$ and $T_t$? Why would this be no use in practice?

### Optimal interpolation

Run the following experiments with the example OI code from the lecture notes:
  1. Try modifying the $\sigma$s, including adding a "real" $\sigma_R$ used to add the observation noise and a "fake" one used in the OI calculation.
  1. Try adding some systematic biases into the observations. How large must these be before the mean square error of the analysis regularly exceeds that of the background?
  2. Try adding the same observation twice, keeping $R$ diagonal. How should $R$ really change, and what difficulties does that introduce?
  
  

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as p
from scipy.linalg import inv


### define the standard deviation of the background and observations

sigma_t = 1.0
sigma_b = 1.0
sigma_r = 1.0

l_t = 0.2
l_e = 0.1
l_b = l_e

s = np.linspace(0, np.pi)

e_b = np.zeros(len(s))
x_t = np.zeros(len(s))

for _ in range(len(s)):
    e_b += np.random.normal(0, sigma_b)*np.exp(-(s-s[_])**2/l_e**2)
    x_t += np.random.normal(0, sigma_t)*np.exp(-(s-s[_])**2/l_t**2)

x_b = x_t + e_b


H = np.zeros((len(s)//5, len(s)))
for _ in range(H.shape[0]):
    H[_,5*_] = 1
    
y = np.dot(H, x_t) 
y += np.random.normal(0, 1, size=(y.shape))


R = sigma_r**2*np.eye(y.shape[0])

s2 = np.broadcast_to(s, (len(s), len(s)))
B = sigma_b**2*np.exp(-(s2-s2.T)**2/l_b**2)

W = B.dot((H.T)).dot(inv(R+H.dot(B.dot(H.T))))

x_a = x_b + W.dot(y-H.dot(x_b))

p.figure(figsize=(15, 6))
p.plot(s, x_t, 'k', label='$x_t$, true state')
p.plot(s, x_b, 'b', label='$x_b$, background guess')
p.scatter(s[::5], y, label='obervation')
p.plot(s, x_a, 'r', label='$x_a$, final analysis')
p.legend();

### 3D - Var

Run the following experiments with the example 3D-Var code from the lecture notes:

 1. Try adding more observations, or some observations of the wind direction as well as speed. Which makes more difference to the analysis accuracy?
 2. Rewrite the cost function in the example to use an iterative solver to calculate $\mathbf{B}^{-1}(\mathbf{x}-\mathbf{x}_b)$ and $\mathbf{R}^{-1}(\mathbf{y}-\mathbf{h}(\mathbf{x}_b))$? Compare the time and memory requirements with the direct inversion method for a few problem sizes.
 3. Experiment with other minimization methods; Can you provide a Jacobian function to speed things up?

In [ ]:
# Example 3D-Var implementation and solution

# We will use some weather-like 2d data and generate the B from climatology.

nx = 26
ny = 11

Lx = 1e6
Ly = 4e5

U_0 =  30.0
radius = 5e4

def wind_field(X, Y, circulations, centres):
    
    U = np.full((ny, nx), U_0)
    V = np.zeros((ny, nx))
    
    for circ, (x, y) in zip(circulations, centres):
        
        r2= (X-x)**2 + (Y-y)**2
        
        u = circ/(2*np.pi)*np.where(r2>radius**2, 1./r2, 1.0/radius**2) 
        
        
        U -= (Y-y)*u
        V += (X-x)*u
        
    return U, V

X, Y = np.meshgrid(np.linspace(0,Lx,nx), np.linspace(0,Ly, ny))


def random_vortices(N, kx=5, ky=5):
    return (200*np.random.lognormal(0, 0.1, size=N)*radius,
            np.random.uniform([-kx*radius, -kx*radius], [Lx+ky*radius, Ly+ky*radius], (N, 2)))

U_t, V_t = wind_field(X, Y, *random_vortices(4, -1, -1))

# observation locations
n_full = 25
n_speed = 25
y_loc = np.random.randint(0, nx*ny, n_full+n_speed)
# observation values
y = np.empty(2*n_full+n_speed)
y[:n_full] = U_t.ravel()[y_loc[:n_full]] + np.random.normal(0, 2.0, n_full)
y[n_full:2*n_full] = V_t.ravel()[y_loc[:n_full]] + np.random.normal(0, 2.0, n_full)
y[2*n_full:] = (np.sqrt(U_t.ravel()[y_loc[n_full:]]**2
                      + V_t.ravel()[y_loc[n_full:]]**2)
                      + np.random.normal(0, 2, n_speed))

def h(x):
    hx = np.empty(2*n_full+n_speed)
    u = x[y_loc]
    v = x[ny*nx+y_loc]
    hx[:n_full] = u[:n_full] = u[:n_full]
    hx[n_full:2*n_full] = v[:n_full]
    hx[2*n_full:] = np.sqrt(u[n_full:]**2+v[n_full:]**2)
    
    return hx

R = 2.0**2*np.eye(2*n_full+n_speed)

U = np.empty((5000,ny,nx))
V = np.empty((5000,ny,nx))

for _ in range(U.shape[0]):
    U[_, : :], V[_, :, :] = wind_field(X, Y, *random_vortices(4))

mu_u = np.mean(U, 0)
mu_v = np.mean(V, 0)

d = np.empty((U.shape[0], 2*ny*nx))
for _ in range(d.shape[0]):
    d[_, :ny*nx] = (U[_, :]-mu_u).ravel() 
    d[_, ny*nx:] = (V[_, :]-mu_v).ravel()
    
B = np.empty((2*nx*ny, 2*nx*ny))
for i in range(2*nx*ny):
    for j in range(2*nx*ny):
        B[i, j] = np.sum(d[:, i]*d[:, j])/(U.shape[0]-1)
        
x_b = np.empty(2*ny*nx)

x_b[:ny*nx] = mu_u.ravel()
x_b[ny*nx:] = mu_v.ravel()

Binv = inv(B)
Rinv = inv(R)

def J(x):
    dx_b = x-x_b
    dx_o = y - h(x)
    return np.dot(dx_b, Binv.dot(dx_b))+np.dot(dx_o,Rinv.dot(dx_o))

def callback(x):
    if False:
        print(J(x))

from scipy.optimize import minimize

res = minimize(J, x_b, method='CG', callback = callback, tol = 1e-3, options={'maxiter':100})
x_a = res.x

U_a = x_a[:ny*nx].reshape((ny,nx))
V_a = x_a[ny*nx:].reshape((ny,nx))

print('J for x_b:', J(x_b))
print('J for x_a:', J(x_a))

 4. Consider the matrix problem

$$\left(
\begin{array}{cc}
\mathbf{R}&\mathbf{H}\\
\mathbf{H}^T &-\mathbf{B}^{-1}
\end{array}
\right)\left(
\begin{array}{c}
\mathbf{a}\\
\mathbf{b}
\end{array}
\right)=
\left(
\begin{array}{c}
\mathbf{c}\\
\mathbf{0}
\end{array}
\right).$$
 Write down the pair of vector equations this gives in non-matrix form. We want to generate an equation in the form $\mathbf{a}=\mathbf{K}\mathbf{c}$, in two different ways:
  - first, solving simultaneously, eliminate \mathbf{b} to get an equation for $\mathbf{a}$, then substitute into the seond equation to get the equation for $\mathbf{b}$
  - second, by eliminate $\mathbf{a}$ between the two equations directly.
By comparing the form of the results, can you see the connection to the lecture?


### Kalman Filter

  1. A hindcast is a "forecast done backwards". That is to say, the model is run backwards so that $\mathbf{x}_{k-1}=\mathbf{M}^{-1}\mathbf{x}_k$. How do the Kalman Filter equations change if the system is run in reverse? In particular, how does the forecast covariance equation change? Modify the Kalman filter example to run backwards.
  3. Modify the code from the lectures:
    - Since the $\mathbf{M}$ matrix is sparse here, create functions to calculate $\mathbf{M}\mathbf{x}$ and $\mathbf{M}^T\mathbf{x}$ directly (a.k.a "matrix-free"), rather than through matrix multiplication or the numpy.dot function. How big does nx need to be for this function to be faster? Can you make your function work on matrices?
    - Modify the code to allow for arbitrary observations at moving locations. Can you find a good strategy to place them?

In [ ]:
# Kalman filter implementation for linearized vortices

nx = 26
ny = 11

Lx = 10000
Ly = 4000

U_0 =  30.0
radius = 500

dt = Lx/(nx-1)/U_0
vortices = random_vortices(200, 100)
def advance_vortices(v):
    v[1][:, 0] +=Lx/(nx-1)

B_k = np.empty((2*ny*nx, 2*ny*nx))
I = np.eye(2*ny*nx)
M = np.zeros((2*ny*nx, 2*ny*nx))
P = np.zeros((2*ny*nx, 2*ny*nx))
Q = np.zeros((2*ny*nx, 2*ny*nx))
Q[::nx,::nx] = B[::nx,::nx] # take forcing error from the statistics for B at x=0


for j in range(ny):
    for i in range(1, nx):
        M[j*nx+i, j*nx+i-1] = 1
        M[ny*nx+j*nx+i, nx*ny+j*nx+i-1] = 1
        
f_b = np.zeros(2*nx*ny)
f_b[::nx] = x_b[::nx]  # boundary condition sets x_a = mean at x=0

n_full = 5
sigma_r = 1.0
y_loc = np.random.randint(0, nx*ny, n_full)
R_k = sigma_r**2*np.eye(2*n_full)
H_k = np.zeros((2*n_full, 2*nx*ny))
for i, j in enumerate(y_loc):
    H_k[i, j] = 1.0
    H_k[n_full+i,nx*ny+j] = 1.0
B_k[:,:] = B


nt = 40

x_a = x_b # initialise with mean values
P[:,:] = B

for _ in range(nt):
    advance_vortices(vortices)
    U_t, V_t = wind_field(X, Y, *vortices)
    x_t = np.concatenate((U_t.ravel(), V_t.ravel())) 
    
    y = H_k.dot(x_t) + np.random.normal(0, sigma_r, size=2*n_full)
    
    # pull forward x
    x_b_k = M.dot(x_a)+f_b
    
    
    # pull forward P to B
    B_k[:,:] = M.dot(P.dot(M.T)) + Q
    # use innovations
    W = B_k.dot((H_k.T)).dot(inv(R_k+H_k.dot(B_k.dot(H_k.T))))
    x_a = x_b_k + W.dot(y-H_k.dot(x_b_k))
    # calculate new P from B
    P[:,:] = (I-W.dot(H_k)).dot(B_k)

### 4D-Var

1. Consider the situation of running a 4D-Var assimilation for a discrete linear model, $\mathbf{M}$, and linear observation operator $\mathbf{H}$ taken over a time window of one timestep with observations only on at the end of the window, so that
$$ \mathbf{x}_1 = \mathbf{M}\mathbf{x}_0,$$
$$\mathcal{J} = \frac{1}{2}(\mathbf{x}_0-\mathbf{x}_b)^T\mathbf{B}^{-1}(\mathbf{x}_0-\mathbf{x}_b) + \frac{1}{2}(\mathbf{y}_1-\mathbf{H}\mathbf{x}_1)^T\mathbf{R}^{-1}(\mathbf{y}_1-\mathbf{H}\mathbf{x}_1).$$
By direct substitution or otherwise, give the relevant optimal value of $\mathbf{x}_1$, in the form $\mathbf{x}_1 = \mathbf{M}\mathbf{x}_b +\mathbf{W}(\mathbf{y}-\mathbf{H}\mathbf{M}\mathbf{x}_b).$. By comparing the optimal interpolation equation or linear 3D-Var solution, can you state the equivalent forecast error covariance matrix being used to assimilate $y_1$? How does this compare to the forward and hindcasting Kalman filter equations?
2. Modify code from the lectures:
  - vary the diffusion coefficient and advection speed in the forward and adjoint models. Which has more influence on the error? 
  - try setting inconsistent values in the adjoint model versus the forward model


In [ ]:
# 4D var implementation for a diffusion problem with swarth observational data

nx = 15
nt = nx**2
dt = 0.1
dx = 1
u = 0.2
k = 2.0

def forward_model(x0):
    x = np.empty((nt, nx, nx))
    x[0, :, :] = x0
    
    for t in range(1, nt):
        x[t, ...] = x[t-1, ...]
        x[t, 1:, :] += u*dt/dx*(x[t-1, :-1, :]-x[t-1, 1:, :])
        x[t, :-1, :] += k*dt/dx**2*(x[t-1, 1:, :]**2-x[t-1, :-1,:]**2)
        x[t, :, :-1] += k*dt/dx**2*(x[t-1, :, 1:]**2-x[t-1, :, :-1]**2)
        x[t, :, 1:] += k*dt/dx**2*(x[t-1, :, :-1]**2-x[t-1, :, 1:]**2)
        x[t, 1:, :] += k*dt/dx**2*(x[t-1, :-1, :]**2-x[t-1, 1:, :]**2)
        
    return x

def adjoint_model(x, y, h):
    l = -h[-1, ...].T.dot(y[-1, :]-h[-1, ...].dot(x[-1,...].ravel())).reshape((nx, nx))
    for t in range(1, nt):
        tmp = l.copy()
        tmp[:-1, :] += u* dt/dx*(l[:-1, :]-l[1:, :])
        z = x[-t-1, ...]*l
        tmp[:-1, :] -= 2.0*k*dt/dx**2*(z[:-1, :]-z[:-1,:])
        tmp[:, :-1] -= 2.0*k*dt/dx**2*(z[:, 1:]-z[:, :-1])
        tmp[:, 1:] -= 2.0*k*dt/dx**2*(z[:, :-1]-z[:, 1:])
        tmp[1:, :] -= 2.0*k*dt/dx**2*(z[:-1, :]-z[1:, :])
        l = tmp - h[-t-1, ...].T.dot(y[-t-1, :]-h[-t-1, ...].dot(x[-t-1,...].ravel())).reshape((nx, nx))
        
    return l
                
    
X, Y = np.meshgrid(np.linspace(0,nx-1,nx), np.linspace(0, nx-1, nx))
x0 =np.exp(-((X-7)**2+(Y-4)**2)/4.0)
x_t = forward_model(x0)


X = X.ravel()
Y = Y.ravel()

Binv = np.empty((nx*nx, nx*nx))
for i in range(nx*nx):
    for j in range(nx*nx):
        d2 = (X[i]-X[j])**2
        dx += (Y[i]-Y[j])**2
        Binv[i, j] = 0.05*np.exp(-0.1*d2)
        

h = np.zeros((nt, 1, nx**2))
y = np.zeros((nt, 1))
x_o = np.zeros((nx, nx))

for _ in range(nt):
    h [_, 0, (_%nx)*nx+_//nx] = 1
    y[_, :] = h[_, :, :].dot(x_t[_, ...].ravel())
    x_o += (h[_, :, :].T.dot(h[_, :, :].dot(x_t[_, ...].ravel()))).reshape((nx, nx))

x_b = np.zeros((nx*nx))


def j(x0, y, h):
    x = forward_model(x0.reshape(nx, nx))
    j_b0 = 0.5*(x0-x_b).dot(Binv.dot((x0-x_b)))
    j_b = j_b0
    for _ in range(nt):
        j_b += 0.5*np.sum((y[_, :]-h[_, ...].dot(x[_,:, :].ravel()))**2)
    return j_b

def jac(x0, y, h):
    x = forward_model(x0.reshape(nx, nx))
    jac= Binv.dot(x0-x_b) + adjoint_model(x, y, h).ravel()
    return jac

from scipy.optimize.linesearch import line_search_armijo as line_search

x_a = x_b.copy()
j0 = j(x_a, y, h)
for _ in range(100):
    pk = -jac(x_a, y, h)
    res = line_search(j, x_a, pk, -pk, j0, args=(y, h), alpha0=0.5)
    x_a += res[0]*pk
    j0 = res[2]
    if res[0]<0.01:
        break

j0 = jac(x_b, y, h)

print('Cost function for $x_b$', j(x_b, y, h))
print('Cost function for $x_a$', j(x_a, y, h))


x_f = forward_model(x_a.reshape((nx, nx)))

p.figure()
p.pcolormesh(x_b.reshape((nx, nx)), vmin=0, vmax=1)
p.colorbar()

p.figure()
p.pcolormesh(x_f[0, ...])
p.colorbar()

p.figure()
p.pcolormesh(x_t[0, ...])
p.colorbar();